In [ ]:
import os
import requests
import pandas as pd
from tqdm import tqdm
import time
import ast

# --------------------------------------------------------------------
# File paths
# --------------------------------------------------------------------
coord_path = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\structure_coordinates.csv"
out_dir = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data"
os.makedirs(out_dir, exist_ok=True)

# --------------------------------------------------------------------
# Load coordinates
# --------------------------------------------------------------------
df = pd.read_csv(coord_path)

def parse_coord(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else x
    except:
        return None

df["COORDINATES"] = df["COORDINATES"].apply(parse_coord)

# Validate rows
df = df[df["COORDINATES"].notna()].copy()

# --------------------------------------------------------------------
# Quiet safe request wrapper
# --------------------------------------------------------------------
def safe_get(url, params=None):
    try:
        r = requests.get(url, params=params, timeout=10)
        if r.status_code == 200:
            return r.json()
        else:
            print(f"\n❌ HTTP {r.status_code}, params={params}")
            return None
    except Exception as e:
        print(f"\n❌ REQUEST ERROR: {e}, params={params}")
        return None

# --------------------------------------------------------------------
# Prepare output list
# --------------------------------------------------------------------
rows = []

print("\nStarting USGS Design Maps queries...\n")

# --------------------------------------------------------------------
# Loop over all coordinates with tqdm
# --------------------------------------------------------------------
for _, row in tqdm(df.iterrows(), total=len(df), desc="Design Maps API"):
    sid = row["STRUCTURE_ID"]
    lat, lon = row["COORDINATES"]

    url = "https://earthquake.usgs.gov/ws/designmaps/asce7-16.json"
    params = {
        "latitude": lat,
        "longitude": lon,
        "riskCategory": "II",
        "siteClass": "D",
        "title": "Earthquake"
    }

    data = safe_get(url, params)

    # Prepare default result row
    entry = {
        "STRUCTURE_ID": sid,
        "COORDINATES": (lat, lon),
        "PGA": None,
        "SS": None,
        "S1": None,
        "SMS": None,
        "SDS": None,
        "SDCS": None,
        "PGAM": None,
        "FPGA": None
    }

    if data and "response" in data and "data" in data["response"]:
        d = data["response"]["data"]
        entry.update({
            "PGA": d.get("pga"),
            "SS": d.get("ss"),
            "S1": d.get("s1"),
            "SMS": d.get("sms"),
            "SDS": d.get("sds"),
            "SDCS": d.get("sdcs"),
            "PGAM": d.get("pgam"),
            "FPGA": d.get("fpga"),
        })
    else:
        # Only print errors/tough cases
        print(f"\n⚠ No usable data for STRUCTURE_ID={sid}, lat={lat}, lon={lon}")

    rows.append(entry)

    time.sleep(0.1)  # polite rate limit

# --------------------------------------------------------------------
# Save final CSV
# --------------------------------------------------------------------
df_out = pd.DataFrame(rows)
out_path = os.path.join(out_dir, "design_maps.csv")
df_out.to_csv(out_path, index=False)

print(f"\nSaved Design Maps results:\n{out_path}\n")


In [ ]:
design_map_schema = {
    "PGA": {
        "category": "Peak ground acceleration",
        "type": "numerical",
        "datatype": float,
        "unit": "g",
        "code_map": None
    },
    "PGA_SEMANTIC": {
        "category": "Mapped peak ground acceleration",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "PGA",
        "bin_rules": [
            { "category": "very_low",  "min": 0.0,  "max": 0.10 },
            { "category": "low",       "min": 0.10, "max": 0.20 },
            { "category": "moderate",  "min": 0.20, "max": 0.35 },
            { "category": "high",      "min": 0.35, "max": 0.55 },
            { "category": "very_high", "min": 0.55, "max": None }
        ],
        "code_map": {
            "none": "Mapped peak ground acceleration not available",
            "very_low":  "very low shaking (PGA < 0.10g)",
            "low":       "low shaking (0.10–0.20g)",
            "moderate":  "moderate shaking (0.20–0.35g)",
            "high":      "high shaking (0.35–0.55g)",
            "very_high": "very high shaking (> 0.55g)"
        }
    },
    "SS": {
        "category": "Mapped spectral acceleration (0.2s)",
        "type": "numerical",
        "datatype": float,
        "unit": "g",
        "code_map": None
    },
    "SS_SEMANTIC": {
        "category": "Mapped 0.2-second spectral acceleration",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "SS",
        "bin_rules": [
            { "category": "low",        "min": 0.0,  "max": 0.5  },
            { "category": "moderate",   "min": 0.5,  "max": 1.0  },
            { "category": "high",       "min": 1.0,  "max": 1.5  },
            { "category": "very_high",  "min": 1.5,  "max": 2.0  },
            { "category": "extreme",    "min": 2.0,  "max": None }
        ],
        "code_map": {
            "none": "0.2-second spectral acceleration not available",
            "low":       "low short-period shaking (SS < 0.5g)",
            "moderate":  "moderate short-period shaking (0.5–1.0g)",
            "high":       "high short-period shaking (1.0–1.5g)",
            "very_high": "very high short-period shaking (1.5–2.0g)",
            "extreme":   "extreme short-period shaking (> 2.0g)"
        }
    },
    "S1": {
        "category": "Mapped spectral acceleration (1.0s)",
        "type": "numerical",
        "datatype": float,
        "unit": "g",
        "code_map": None
    },
    "S1_SEMANTIC": {
        "category": "Mapped 1-second spectral acceleration",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "S1",
        "bin_rules": [
            { "category": "low",        "min": 0.0,  "max": 0.2 },
            { "category": "moderate",   "min": 0.2,  "max": 0.4 },
            { "category": "high",       "min": 0.4,  "max": 0.6 },
            { "category": "very_high",  "min": 0.6,  "max": 0.8 },
            { "category": "extreme",    "min": 0.8,  "max": None }
        ],
        "code_map": {
            "none": "1-second spectral acceleration not available",
            "low":       "low long-period shaking (S1 < 0.2g)",
            "moderate":  "moderate long-period shaking (0.2–0.4g)",
            "high":      "high long-period shaking (0.4–0.6g)",
            "very_high": "very high long-period shaking (0.6–0.8g)",
            "extreme":   "extreme long-period shaking (> 0.8g)"
        }
    },
    "SMS": {
        "category": "Site-modified spectral acceleration (0.2s)",
        "type": "numerical",
        "datatype": float,
        "unit": "g",
        "code_map": None
    },
    "SMS_SEMANTIC": {
        "category": "Site-modified 0.2-second spectral acceleration",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "SMS",
        "bin_rules": [
            { "category": "low",        "min": 0.0,  "max": 0.75 },
            { "category": "moderate",   "min": 0.75, "max": 1.25 },
            { "category": "high",       "min": 1.25, "max": 1.75 },
            { "category": "very_high",  "min": 1.75, "max": 2.25 },
            { "category": "extreme",    "min": 2.25, "max": None }
        ],
        "code_map": {
            "none": "Site-modified 0.2-second spectral acceleration not available",
            "low":       "low soil-amplified shaking (SMS < 0.75g)",
            "moderate":  "moderate soil-amplified shaking (0.75–1.25g)",
            "high":      "high soil-amplified shaking (1.25–1.75g)",
            "very_high": "very high soil-amplified shaking (1.75–2.25g)",
            "extreme":   "extreme soil-amplified shaking (> 2.25g)"
        }
    },
    "SDS": {
        "category": "Design spectral acceleration (0.2s)",
        "type": "numerical",
        "datatype": float,
        "unit": "g",
        "code_map": None
    },
    "SDS_SEMANTIC": {
        "category": "Design 0.2-second spectral acceleration",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "SDS",
        "bin_rules": [
            { "category": "low",        "min": 0.0,  "max": 0.33 },
            { "category": "moderate",   "min": 0.33, "max": 0.67 },
            { "category": "high",       "min": 0.67, "max": 1.00 },
            { "category": "very_high",  "min": 1.00, "max": 1.50 },
            { "category": "extreme",    "min": 1.50, "max": None }
        ],
        "code_map": {
            "none": "Design 0.2-second spectral acceleration not available",
            "low":       "low design short-period shaking (SDS < 0.33g)",
            "moderate":  "moderate design short-period shaking (0.33–0.67g)",
            "high":      "high design short-period shaking (0.67–1.00g)",
            "very_high": "very high design short-period shaking (1.00–1.50g)",
            "extreme":   "extreme design short-period shaking (> 1.50g)"
        }
    },
    "SDCS": {
        "category": "Design spectral acceleration (1.0s)",
        "type": "numerical",
        "datatype": float,
        "unit": "g",
        "code_map": None
    },
    "SDCS_SEMANTIC": {
        "category": "Design 1.0-second spectral acceleration",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "SDCS",
        "bin_rules": [
            { "category": "low",        "min": 0.0,  "max": 0.15 },
            { "category": "moderate",   "min": 0.15, "max": 0.30 },
            { "category": "high",       "min": 0.30, "max": 0.50 },
            { "category": "very_high",  "min": 0.50, "max": 0.70 },
            { "category": "extreme",    "min": 0.70, "max": None }
        ],
        "code_map": {
            "none": "Design 1-second spectral acceleration not available",
            "low":       "low design long-period shaking (SD1 < 0.15g)",
            "moderate":  "moderate design long-period shaking (0.15–0.30g)",
            "high":      "high design long-period shaking (0.30–0.50g)",
            "very_high": "very high design long-period shaking (0.50–0.70g)",
            "extreme":   "extreme design long-period shaking (> 0.70g)"
        }
    },
    "FPGA": {
        "category": "Site-modified peak ground acceleration",
        "type": "numerical",
        "datatype": float,
        "unit": "g",
        "code_map": None
    },
    "FPGA_SEMANTIC": {
        "category": "Site-modified peak ground acceleration",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "FPGA",
        "bin_rules": [
            { "category": "very_low",  "min": 0.0,  "max": 0.15 },
            { "category": "low",       "min": 0.15, "max": 0.30 },
            { "category": "moderate",  "min": 0.30, "max": 0.45 },
            { "category": "high",      "min": 0.45, "max": 0.65 },
            { "category": "very_high", "min": 0.65, "max": None }
        ],
        "code_map": {
            "none": "Site-modified peak ground acceleration not available",
            "very_low":  "very low amplified ground shaking (FPGA < 0.15g)",
            "low":       "low amplified ground shaking (0.15–0.30g)",
            "moderate":  "moderate amplified ground shaking (0.30–0.45g)",
            "high":      "high amplified ground shaking (0.45–0.65g)",
            "very_high": "very high amplified ground shaking (> 0.65g)"
        }
    }
}